In [1]:
import csv
import requests
import sys

In [2]:
address = "https://prometheus.nautilus.optiputer.net"
query = 'kube_node_status_allocatable{resource="nvidia_com_gpu", node="prp-gpu-2.t2.ucsd.edu"}[5m]'

In [4]:
response = requests.get('{0}/api/v1/query'.format(address), params={'query': query})
results = response.json()['data']['result']

In [13]:
import time
print(time.time())

1555957764.9191208


In [12]:
results

[{'metric': {'__name__': 'kube_node_status_allocatable',
   'endpoint': 'https-main',
   'instance': '10.244.111.148:8443',
   'job': 'kube-state-metrics',
   'namespace': 'monitoring',
   'node': 'prp-gpu-2.t2.ucsd.edu',
   'pod': 'kube-state-metrics-7574c6c55-jcw2g',
   'resource': 'nvidia_com_gpu',
   'service': 'kube-state-metrics',
   'unit': 'integer'},
  'values': [[1555956755.694, '8'],
   [1555956785.694, '8'],
   [1555956815.694, '8'],
   [1555956845.694, '8'],
   [1555956875.694, '8'],
   [1555956905.694, '8'],
   [1555956935.694, '8'],
   [1555956965.694, '8'],
   [1555956995.694, '8'],
   [1555957025.694, '8']]},
 {'metric': {'__name__': 'kube_node_status_allocatable',
   'endpoint': 'https-main',
   'instance': '10.244.221.97:8443',
   'job': 'kube-state-metrics',
   'namespace': 'monitoring',
   'node': 'prp-gpu-2.t2.ucsd.edu',
   'pod': 'kube-state-metrics-6cdccb7899-dzhrk',
   'resource': 'nvidia_com_gpu',
   'service': 'kube-state-metrics',
   'unit': 'integer'},
  'v

In [8]:
# Build a list of all labelnames used.
labelnames = set()
for result in results:
    labelnames.update(result['metric'].keys())

# Canonicalize
labelnames.discard('__name__')
labelnames = sorted(labelnames)

In [9]:
labelnames

['endpoint',
 'instance',
 'job',
 'namespace',
 'node',
 'pod',
 'resource',
 'service',
 'unit']

In [11]:
# Write the samples.
with open('result.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['node', 'timestamp', 'values'] + labelnames)
    for result in results:
        l = [result['metric'].get('__name__', '')] + result['values']
        for label in labelnames:
            l.append(result['metric'].get(label, ''))
        writer.writerow(l)